описание

In [27]:
import json
import pandas as pd
from sklearn.cluster import DBSCAN
import ctypes
import lasio
from welly import Well, Project
import os

In [28]:
!ls *.LAS

['/Users/victorpotysyev/PycharmProjects/lqc_maker/1607/1607_18902381.las', '/Users/victorpotysyev/PycharmProjects/lqc_maker/1607/1607_18902341.las', '/Users/victorpotysyev/PycharmProjects/lqc_maker/1607/1607_18902421.las', '/Users/victorpotysyev/PycharmProjects/lqc_maker/1607/1607_18902401.las', '/Users/victorpotysyev/PycharmProjects/lqc_maker/511/511R_511-P_ak.las', '/Users/victorpotysyev/PycharmProjects/lqc_maker/511/511R_511-P_vik.las', '/Users/victorpotysyev/PycharmProjects/lqc_maker/511/511R_511_bk.las', '/Users/victorpotysyev/PycharmProjects/lqc_maker/511/511R_S-Koms_511p_rk.las', '/Users/victorpotysyev/PycharmProjects/lqc_maker/511/511R_511_vik.las', '/Users/victorpotysyev/PycharmProjects/lqc_maker/511/511R_511-P_rk.las', '/Users/victorpotysyev/PycharmProjects/lqc_maker/511/511R_511_ggk_cav.las', '/Users/victorpotysyev/PycharmProjects/lqc_maker/511/511R_ЯМК Северо-Комсомольское 511Р.las', '/Users/victorpotysyev/PycharmProjects/lqc_maker/511/511R_S-Koms_511p_ak.las', '/Users/vict

In [ ]:
#словари перенести в json файлы
ds="LQC"
distance_typing = 1000        #параметр для настройки типизации интервалов записи

famdict={'CALI':     ["Caliper","mm"],
         'GR':       ["Gamma Ray","Gapi"],
         'RHOB':     ["Bulk Density", "g/cm3"],
         'NPHI':     ["Neutron Porosity", "v/v"],
         'PEF':      ["Photoelectric Factor", "b/e"],
         'IL':       ["Induction Resistivity", "Ohmm"],
         'IL1':      ["Induction Resistivity", "Ohmm"],
         'IL2':      ["Induction Resistivity", "Ohmm"],
         'IL3':      ["Induction Resistivity", "Ohmm"],
         'IL4':      ["Induction Resistivity", "Ohmm"],
         'IL5':      ["Induction Resistivity", "Ohmm"],
         'RTIL':     ["Formation Resistivity", "Ohmm"],
         'LL':       ["Lateral Log Resistivity", "Ohmm"],
         'LL1':      ["Lateral Log Resistivity", "Ohmm"],
         'LL2':      ["Lateral Log Resistivity", "Ohmm"],
         'LL3':      ["Lateral Log Resistivity", "Ohmm"],
         'LL4':      ["Lateral Log Resistivity", "Ohmm"],
         'LL5':      ["Lateral Log Resistivity", "Ohmm"],
         'RTLL':     ["Formation Resistivity", "Ohmm"],
         'RT':       ["Formation Resistivity", "Ohmm"],
         'SP':       ["Spontaneous Potential", "mV"],
         'DTP':      ["Compressional Slowness", "us/m"],
         'DTS':      ["Shear Slowness", "us/m"],
         'MPHI':     ["NMR Porosity", "v/v"],
         'NKTB':     ["Neutron Far", "UE"]}

lqcdict={'CALI':     ['DS','CALI','ДС'],
         'GR':       ['GK','ГК', 'GR'],
         'RHOB':     ['GGK','RHOB','ГГКп'],
         'NPHI':     ['WS','TRNP','W'],
         'PEF':      ['PEF'],
         'IL':       ['ИК','IK'],
         'IL1':      ['RO05','IVR1'],
         'IL2':      ['RO07','IVR2'],
         'IL3':      ['RO10','IVR3'],
         'IL4':      ['RO14','IVR4'],
         'IL5':      ['RO20','IVR5'],
         'RTIL':     ['RT_IK'],
         'LL':       ['BK','БК'],
         'LL1':      ['L1FC','BK1'],
         'LL2':      ['L2FC', 'BK2'],
         'LL3':      ['L3FC', 'BK3'],
         'LL4':      ['L4FC', 'BK4'],
         'LL5':      ['L5FC', 'BK5'],
         'RTLL':     ['RT_BK', 'REK_BK'],
         'RT':       ['ROP', 'RT'],
         'SP':       ['PS', 'SP'],
         'DTP':      ['DTp','DTP', 'DT','DTCO'],
         'DTS':      ['DTs','DTs','DTSM'],
         'MPHI':     ['TCMR'],
         'NKTB':     ['CFTC','НКТб','NKTB']}


In [ ]:
def datasetlist_sort(well, datasetlist):
	dsets = []
	for dset in datasetlist:
		top = db.variableLoad(well, dset,db.referenceName(well, dset))[0]
		bottom = db.variableLoad(well, dset,db.referenceName(well ,dset))[-1]
		if bottom > db.referenceValue(well, "Index" ,db.datasetSize(well, "Index") - 1):
			print("Dataset {} larger than Index!".format(dset))
			ctypes.windll.user32.MessageBoxW(0, "Dataset {} larger than Index!".format(dset), "Index Error", 0)
		dsets.append([dset, top, bottom, bottom-top])
	ds_df = pd.DataFrame(dsets, columns=['dataset', 'top', 'bottom', 'size'])
	ds_df.sort_values(by=['top', 'size'], inplace=True)

	dbs = DBSCAN(eps=distance_typing, min_samples=1).fit(ds_df[['top', 'bottom', 'size']])
	ds_df['labels'] = dbs.labels_

	print(ds_df)

	return ds_df[['dataset', 'labels']].values.tolist()



In [ ]:
well_list = db.selectedWellList()
for well in well_list:
	print(well)
	if  db.datasetExists(well, ds):  #надо сделать индикатор наличия лкуси датасета для дальнейшего использования.
		db.datasetDelete(well, ds)

	if db.datasetExists(well,'Index'):
		MD = [round(d*0.1, 1) for d in range(0, int(round(db.referenceValue(well, "Index", db.datasetSize(well, "Index") - 1) / 0.1, 0)) + 1)]
		db.datasetCreate(well, ds ,"MD", "Measured Depth", "m", MD)
	else:
		print("No Index dataset in well DB!")
		break

	for data_set, dslabel in datasetlist_sort(well, db.selectedDatasetList(well)):
		if dslabel == 0:
			dslabel_ = ""
		else:
			dslabel_ = "_" + str(dslabel)
		for cname in db.variableList(well, data_set):
			if  any(cname in sublist for sublist in list(lqcdict.values())) : # есть ли кривая в словаре lqcdict гделибо
				lqcname = list(lqcdict.keys())[list(lqcdict.values()).index([sublist for sublist in list(lqcdict.values()) if cname in sublist][0])] + dslabel_ #выбираем из словаря ключ соотвествующий найденному в значениях имени кривой

				#версия кривой
				if db.variableExists(well, ds, lqcname):
					clist = [i for i in db.variableList(well, ds) if i.find(lqcname) != -1]
					if len(clist)>=1:
						lqcname=lqcname+"_"+str(len(clist))

				db.variableCopy(well, data_set, cname, ds, lqcname)
				db.variableHistoryAdd(well, ds, lqcname, "Curve copied to LQC from dataset {}".format(data_set))

				#обновление family
				if lqcname.find("_") == -1:
					version_index = len(lqcname)
				else:
					version_index = lqcname.find("_")
				if db.variableFamily(well, ds, lqcname) != famdict[lqcname[0:version_index]][0]:
					db.variableFamilyChange(well, ds, lqcname, famdict[lqcname[0:version_index]][0])

				#обновление unit для некоторых кривых
				if db.variableUnit(well, ds, lqcname)!=famdict[lqcname[0:version_index]][1]:
					crunit = ""

					if lqcname[0:version_index] == "NPHI": #добавить проверку по конкретным методам ГК НК
						if  float(db.variableInformation(well, ds, lqcname," Mean"))<1:
							crunit = "v/v"
						else:
							crunit = "%"

					elif lqcname[0:version_index] == "GR" :
						if float(db.variableInformation(well, ds, lqcname, "Mean"))<50:
							crunit = "uR/H"
						else:
							crunit = "Gapi"

					elif lqcname[0:version_index] == "CALI" :
						if float(db.variableInformation(well, ds, lqcname, "Mean"))<1:
							crunit = "m"
						else:
							crunit = "mm"
					else:
						crunit = famdict[lqcname[0:version_index]][1]

					db.variableUnitChange(well, ds, lqcname, crunit)



In [39]:
#указать папку с ласами они могут быть из разных скважин
las_path = "\\511"

#используя lasio скачать данные и передать в функцию для создания рабочего датасета

0it [00:00, ?it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)
1it [00:00,  3.75it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)
2it [00:00,  5.76it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)
3it [00:00,  4.55it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)
4it [00:00,  4.45it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: U

1607
1607
1607
1607
511R
511R
511R
511R
511R
511R
511R
511R
511R
511R
511R
511R
511R
511R
511R
511R
511R
511R
